In [99]:
import pandas as pd
import numpy as np
import json
import altair as alt
import os
#os.chdir("..")


Reading in the multisheet dataset

In [100]:
dfs = pd.read_excel("data/raw/ons_subnational.xlsx", None, skiprows=0)
cleanded_dfs = []
for df in dfs.values():
    # itterate down the rows until a row with 'Area Code' in the first column is found
    # this is the start of the data
    for i in range(len(df)):
        if df.iloc[i,0] == 'Area Code':
            df.columns = df.iloc[i]
            df = df.iloc[i+1:]
            cleanded_dfs.append(df)
            break

Concatting, tidying and melting:

In [101]:
df = pd.concat(cleanded_dfs)

id_vars = ['Area Code', 'Country', 'Nation', 'Region',
       'County or Unitary Authority', 'Local Authority District', 'Lower 95% Confidence Interval',
       'Upper 95% Confidence Interval', 'Notes',
       'ITL Level 2', 'ITL Level 3', 
       'Data accuracy']


df = df.melt(id_vars=id_vars, var_name='Series', value_name='Value')


merge in the NHS data:

In [102]:
# if Local Authority District is undefined, use County or Unitary Authority
df['Local Authority District'] = df['Local Authority District'].fillna(df['County or Unitary Authority'])

df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
df = df.dropna(subset=['Local Authority District', 'Value'])
df = df.rename(columns={'Local Authority District': 'name'})

# merge in the NHS
nhs_df = pd.read_csv("data/cleaned/nhs_ae.csv")
nhs_df = nhs_df.rename(columns={'id': 'Area Code', 'value': 'Value', 'series': 'Series'})

df = pd.concat([df, nhs_df])

df['median'] = df.groupby(['Series'])['Value'].transform('median')
df['std'] = df.groupby(['Series'])['Value'].transform('std')
df['z_score'] = (df['Value'] - df['median']) / df['std']

df['norm_value'] = df.Value/df['median']

series_to_keep = {
    "Gross Value Added (GVA) per hour worked (£)" : {
              "Theme": "Productivity",
              "name": "GVA per hour worked (£)",
              "format": "$,.2f",
              "scheme" : "interpolateWarm",
              "geo": "LAD.geojson",
              "geo_key" : "LAD21CD", # the key in properties to match on
              "geo_name" : "LAD21NM" # the key for the name of the area
       },
    'Gross median weekly pay (£)' : {
              "Theme": "Income",
              "name": "Gross median weekly pay (£)",
              "format": "$,.2f",
              "geo": "LAD.geojson",
              "geo_key" : "LAD21CD",
              "geo_name" : "LAD21NM"
       },
    'Employment rate, ages 16-64 (%)' : {
              "Theme": "Employment",
              "name": "Employment rate, ages 16-64 (%)",
              "format": ".2%",
              "geo": "LAD.geojson",
              "geo_key" : "LAD21CD",
              "geo_name" : "LAD21NM",
              "multiplier" : 0.01
       },
    'Average travel time in minutes to reach nearest large employment centre (500 to 4999 jobs available), by public transport or walking (minutes)' : {
              "Theme": "Transport",
              "name": "Average Public Transport Travel Time to Nearest Large Employment Centre (minutes)",
              "format": ".2f",
              "geo": "LAD.geojson",
              "geo_key" : "LAD21CD",
              "geo_name" : "LAD21NM"
       },
    'Percentage of young people achieving GCSEs (and equivalent qualifications) in English and maths by age 19 (%)' : {
              "Theme": "Education",
              "name": "Percentage of Young People Achieving GCSEs in English and Maths by Age 19 (%)",
              "format": ".2%",
              "geo": "LAD.geojson",
              "geo_key" : "LAD21CD",
              "geo_name" : "LAD21NM",
              "multiplier" : 0.01
       },
       '0_on_target': {
              "Theme": "Health",
              "name" : "Percentage of A&E Attendances Admitted, Transferred or Discharged Within 4 Hours (August 2023)",
              "format" : ".2%",
              #"scheme" : "interpolateViridis",
              "geo": "nhs_uk_harmonised.geojson",
              "geo_key" : "id",
              "geo_name" : "name",
       },
"""     'Female Healthy Life Expectancy (years)' : {
              "Theme": "Health",
              "name" : "Female Healthy Life Expectancy (years)",
              "format" : ".2f",
              "geo": "LAD.geojson",
              "geo_key" : "LAD21CD",
              "geo_name" : "LAD21NM"
       },
    'Male Healthy Life Expectancy (years)' : {
              "Theme": "Health",
              "name" : "Male Healthy Life Expectancy (years)",
              "format" : ".2f",
              "geo": "LAD.geojson",
              "geo_key" : "LAD21CD",
              "geo_name" : "LAD21NM"
    },
 """    'Age-standardised mortality rate for those aged under 75 (per 100,000 population)': {
              "Theme": "Health",
              "name" : "Under 75 Age-standardised mortality rates",
              "format" : ".2f",
              "geo": "LAD.geojson",
               "geo_key" : "LAD21CD",
              "geo_name" : "LAD21NM"
       },
    'Mean satisfaction with your life nowadays scored 0 (not at all) - 10 (completely)': {
              "Theme": "Wellbeing",
              "name" : "Mean Reported Life Satisfaction (0-10)",
              "format" : ".2f",
              "geo": "LAD.geojson",
               "geo_key" : "LAD21CD",
              "geo_name" : "LAD21NM"
       },
    'Net additions per 1,000 stock': {
              "Theme": "Housing",
              "name" : "Net additions per 1,000 stock",
              "format" : ".2f",
              "geo": "LAD.geojson",
              "geo_key" : "LAD21CD",
              "geo_name" : "LAD21NM"
       }
}

df = df[df.Series.isin(series_to_keep.keys())]
df["geo"] = df.Series.apply(lambda x: series_to_keep[x]['geo'])

df['multiplier'] = df.Series.apply(lambda x: series_to_keep[x]['multiplier'] if 'multiplier' in series_to_keep[x] else 1)
df['Theme'] = df.Series.apply(lambda x: series_to_keep[x]['Theme'])
df['format'] = df.Series.apply(lambda x: series_to_keep[x]['format'])
df['scheme'] = df.Series.apply(lambda x: series_to_keep[x]['scheme'] if 'scheme' in series_to_keep[x] else 'interpolateCool')
df['geo_key'] = df.Series.apply(lambda x: series_to_keep[x]['geo_key'])
df['geo_name'] = df.Series.apply(lambda x: series_to_keep[x]['geo_name'])
df.Series = df.Series.apply(lambda x: series_to_keep[x]['name'])
df['series_id'] = df['Series'].astype('category').cat.codes

df['rank'] = df.groupby(['Series'])['Value'].rank(ascending=False)
df['out_of'] = df.groupby(['Series'])['Value'].transform('count')

# mad code
import numpy as np
def mad(x):
    return np.median(np.abs(x - np.median(x))) * 1.4826

df['MAD'] = df.groupby(['Series'])['Value'].transform(mad)

df['median'] = df.groupby(['Series'])['Value'].transform('median')

df['norm_value'] = (df['Value'] - df['median']) / df['MAD']

df['rank'] = df.groupby(['Series'])['Value'].rank(ascending=False)

# sort according to the order in series_to_keep
df = df.sort_values(by=['series_id', 'rank'])

# multiply by a multiplier if a multiplier is defined
df['Value'] = df.apply(lambda x: x['Value'] * x['multiplier'] if 'multiplier' in x else x['Value'], axis=1)

df[["Area Code", "name", "Series", "Theme", "Value", "median", "z_score", "norm_value", "rank", "out_of", "series_id", "format", "scheme", "geo", "geo_key", "geo_name"]].to_json("/Users/finn/Documents/GitHub/eco4Map/data/cleaned/indicators.json", orient="records") 


In [106]:
df.norm_value.quantile(0.95)

2.3347757058805088

In [50]:
temp_df = df.query("Series == 'Percentage of A&E Attendances Admitted, Transferred or Discharged Within 4 Hours'")
temp_df[temp_df['Area Code'].str[0]=='S']['Area Code'].unique()

array([], dtype=object)

In [37]:
# in geoJSON
in_geoJSON = ['S08000028', 'S08000025', 'S08000022', 'S08000026', 'S08000017','S08000015', 'S08000020', 'S08000024', 'S08000016', 'S08000019','S08000019', 'S08000016', 'S08000024', 'S08000020', 'S08000015','S08000017', 'S08000026', 'S08000022', 'S08000025', 'S08000028']
in_ons     = ['S08000028', 'S08000025', 'S08000022', 'S08000026', 'S08000017','S08000015', 'S08000020', 'S08000024', 'S08000016', 'S08000019']
# in 

In [40]:
df.Series.unique()

array(['GVA per hour worked (£)', 'Gross median weekly pay (£)',
       'Employment rate, ages 16-64 (%)',
       'Average Public Transport Travel Time to Nearest Large Employment Centre (minutes)',
       'Percentage of Young People Achieving GCSEs in English and Maths by Age 19 (%)',
       'Female Healthy Life Expectancy (years)',
       'Male Healthy Life Expectancy (years)',
       'Under 75 Age-standardised mortality rates',
       'Mean Reported Life Satisfaction (0-10)',
       'Net additions per 1,000 stock'], dtype=object)

In [21]:
df = cleanded_dfs[5]
df

,Area Code,Country,ITL Level 1,ITL Level 2,Combined Authority or City Region,Total FDI international investment position abroad at end period (£ million),Notes
1,K02000001,United Kingdom,NaN,NaN,NaN,1769316,NaN
2,K99000001,NaN,UK not allocated,NaN,NaN,2597,NaN
3,TLC,NaN,North East,NaN,NaN,26878,NaN
4,TLC1,NaN,NaN,Tees Valley and Durham,NaN,16413,NaN
5,E47000006,NaN,NaN,NaN,Tees Valley,3249,NaN
...,...,...,...,...,...,...,...
64,na,NaN,NaN,NaN,Edinburgh and South East Scotland City Region,24227,NaN
65,TLM8,NaN,NaN,West Central Scotland,NaN,27271,NaN
66,na,NaN,NaN,NaN,Glasgow City Region,27470,NaN
67,TLM9,NaN,NaN,Southern Scotland,NaN,5742,NaN
